In [1]:
import pandas as pd
from google.cloud import bigquery
from datetime import datetime, timedelta
from google.cloud.bigquery import job
PROJECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJECT)


# https://www.notion.so/hecto1app/2-1f90b8e5ab1680f69e1ce36a566995ad 처리건
# Query-64

c:\Users\Owner\miniconda3\envs\stats\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:


query = """
select * from ballosodeuk.ynam.rfm_table_byshop_category_power
"""
df = bq.query(query).to_dataframe()

In [16]:
query = """ 
with tg as 
  (select b.wk_id as user_id
  from
    (select distinct member_no
    from `dw.fact_shopby_reward`
    where 
      -- register_dt >= current_date() -31 
      -- register_dt between "2025-04-19" and "2025-05-18"
      register_dt between "2025-04-19" and "2025-05-18"
      and reason_detail like "[챌린지]%") a
  inner join `dw.dim_shopby_member` b on a.member_no = b.member_no)

,ft_tg as 
  (select user_id, current_shoji
  from
    (select 
      user_id, current_shoji,
      row_number() over (partition by user_id order by register_dt desc) as rn
    from `dm.agg_user_cash_daily`
    where signout_dt is null and user_id in (
      select user_id from tg ))
  where rn = 1)


,chg_joined as (
  select distinct user_id
  from dw.fact_airbridge_event
    ,unnest(event_detail) ed with offset as offset_ed
  where 1=1
    and event_dt between "2025-03-19" and "2025-04-19"
    and event_category = 'view_get__ch_join_done (App)'
    and event_action in ("1163", "1161", "1107")
)

,fn_tg as (
  select *
  from ft_tg
  where user_id not in (
    select user_id
    from chg_joined
  )
)

,shoji_range_df as
  (select user_id,
    case 
      when current_shoji is Null then 'No-shoji'
      when current_shoji = 0 then '0'
      when current_shoji between 1 and 999 then '1~999'
      else 
        concat(
            cast(floor(current_shoji / 1000) * 1000 as int64),
            '~',
            cast(floor(current_shoji / 1000) * 1000 + 999 as int64)
        ) 
    end as shoji_range, 
    case 
      when current_shoji is Null then null
      when current_shoji = 0 then 0
      when current_shoji between 1 and 999 then 999
      else cast(floor(current_shoji / 1000) * 1000 + 999 as int64)
    end as max_shoji_range,
    current_shoji
  from fn_tg)


select max_shoji_range, shoji_range, user_id
from shoji_range_df
where max_shoji_range between 1999 and 9999
order by max_shoji_range

"""

In [17]:
df = bq.query(query).to_dataframe()
df.head()

,max_shoji_range,shoji_range,user_id
0,1999,1000~1999,8d3a034c-e28d-41ad-a4de-a73e3635d917
1,1999,1000~1999,ba73f753-a829-40ce-9082-fba110d250ff
2,1999,1000~1999,e7ab2217-ad5e-4eb0-a68e-cb8dedc06698
3,1999,1000~1999,f9303ae9-e6bc-4dc7-9b6b-8f662186abcd
4,1999,1000~1999,3cb0fc47-f7cb-4759-8bd8-3d9c4315871c


In [19]:
df.to_excel("C:\\Users\\Owner\\Documents\\업무 문서\\1. Project\\250520_추출_DI-TASK-638\\250516_get_rcvr_raw.xlsx")

In [18]:
df.user_id.nunique()

36724